# HW2 - RL Answers

Q1.Examining Dataset

In [ ]:
#library(tidyverse)

In [ ]:
#I stored the data into a new variable 'df' for practical use.
df <- USArrests

In [ ]:
#As part of the data preparation, I’m going to create a new column ‘State’ from the row names.
df = cbind(State = row.names(df) , df)

#Make the state column a factor variable.
df = transform(df, State=factor(State))

#Examining the first 5 rows.
head(df,5)

In [ ]:
#Examining the last 5 rows.
tail(df, 5)

In [ ]:
# Number of rows (observations) 
nrow(df)

In [ ]:
# Number of columns (variables)
ncol(df)

In [ ]:
#Data exploration
summary(df)
str(df)

In [ ]:
#I stored my dependent and independent variables for practical use.
Mur <- df$Murder
Ass <- df$Assault

Q2.Plot your variables from your dataset and show the mean point as red.

In [ ]:
#Plotted my independent and dependent variables and stored as 'plot_var'
plot_var <- ggplot(df, aes(Assault, Murder)) + geom_point()
#I calculated my mean values and stored them in their coordinates
x <- mean(Mur)
y <- mean(Ass)
#Plotted my mean value and stored as 'plot_mean'
plot_mean <- geom_point(aes(y,x), colour='red')
#Combined them
plot_var + plot_mean

Q3.Calculate the c and m values

In [ ]:
fit <- lm(Murder ~ Assault, data = df)
#Finding coefficients
as.formula(
  paste0("y ~ ", round(coef(fit),3), " + ", 
    paste(sprintf("%.2f * %s", 
                  coef(fit)[-1],  
                  names(coef(fit)[-1])), 
          collapse=" + ")
  )
)
#or examining the summary()
summary(fit)

Q4.In brief, find out the predicted values of Y for corresponding values of X using the linear equation

In [ ]:
#Write a function to use the Y=m*X*c
lin_eq <- function(x){
    y=(0.63)*x + 0.042
}
#Use apply to execute the function on your X values
pred<-lapply(Ass,lin_eq)
#Converting list to a numeric vector.
a<-unlist(pred)
class(a)

Q5.

In [ ]:
#Firstly, I drew the regression line and previous values as geom_point(), 
#Then I added my mean value from previous exercise(plot_mean),
#Then I added my predicted values using geom_point(aes(a)) and made them green 
p<-ggplot(df,aes(Assault, Murder)) + geom_smooth(method='lm',se=FALSE,formula=y~x) + geom_point() + plot_mean + geom_point(aes(a),colour='green') 
lm_eq <- function(df){
    
    m <- lm(Murder ~ Assault, df);

    eq <- substitute((y) == a + b %.% (x), 

         list(a = format(unname(coef(m)[1]), digits = 2),

              b = format(unname(coef(m)[2]), digits = 2)))


    as.character(as.expression(eq));

}
p1 <- p + geom_text(x = 80, y = 17, label = lm_eq(df), parse = TRUE)

p1


Q6.Calculate the sum of squared errors 

In [ ]:
SSTotal <- var(Mur) * (nrow(df)-1)
SSE <- sum(fit$resid^2 )
SSreg <- SSTotal - SSE
SSreg

Outliers

In [ ]:
#Q7.Create box plots of your variables.
par(mfrow=c(1,2))
boxplot(Mur, xlab = "Murder")
boxplot(Ass, xlab = "Assault")

In [ ]:
#Q8.Find the value and the index of this outlier.
#I didn't have a column consists of an outlier value, but if it did I would do the following.
#out_val <- boxplot.stats(df$outliercolunm)$out
#print(paste("Outlier values:",out_val))
#out_ind <- which(df$outliercolumn %in% c(out_val))
#print(paste("Outlier indices:",out_ind))

Model Building 1

In [ ]:
#Q9.Build a model with your variables
Model <-lm(Mur~Ass, data=df) 
#Give the summary of the model
summary(Model)

In [ ]:
#Q10.Write a function that does the beta coefficients calculation.
lin_eq <- function(x){
    y=(0.63)*x + 0.042
}
#Use apply to execute the function on your Y values
pred_2<-lapply(Ass,lin_eq)
head(pred_2)

Train – Test

In [ ]:
#Q11.Split your dataset as we did in the class

#set starting point
set.seed(10)

#spliting the dataset
trainingRowIndex <- sample(1:nrow(df),0.8*nrow(df))

#model training data
trainingData <- df[trainingRowIndex,] 

#model testing data
testingData <- df[-trainingRowIndex,] 

Model Building-2

In [ ]:
#Q13.Build a model with your training data and give the summary of your model.
linModel <-summary(lm(Murder~Assault, data=trainingData))
linModel

In [ ]:
#Q14.What is the degree of freedom and how does it calculate?
linModel$fstatistic[3] 

Model Fitting

In [ ]:
#Q15.Fit your model and predict for your testing data
#Recreating the model 
lin_Model <-lm(Murder~Assault, data=trainingData ) 
murPredict <- predict(lin_Model, testingData)
actual_preds <- data.frame(cbind(actuals=testingData$Murder, predicted=murPredict))
head(actual_preds)

In [ ]:
#Q16.Find the correlation accuracy
correlation_accuracy <- cor(actual_preds)
correlation_accuracy

In [ ]:
#Q17.Plot your data with your regression line (the model you build with your training data).
plot(Murder~Assault, df)
abline(lin_Model,col=3,lwd=2)